##Student Number:Mahdi Saieedi

##Name:401207254

#**Notes**
In this notebook you are going to implement [NES](https://arxiv.org/pdf/1804.08598.pdf) blackbox attack and test in on Cifar10. First, you must implement the provided functions. Then you must test the attack and report the average number of queries and the success rate of the attack. You can define as many additional functions as you want. If you want to alter the structure of the provided code, please do as minimally as possible.

#**Dependencies**

In [1]:
from google.colab import drive
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from typing import Type
import numpy as np
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from torchvision.models import resnet18

#**Defining the Model and Dataloader**
Here the link to a ResNet18 checkpoint is provided. Please use this checkpoint.

In [2]:
batch_size = 128
transform = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='/content/cifar10/', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root='/content/cifar10/', train = False, download = True, transform = transforms.Compose([transforms.ToTensor(),]))
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class ResNet18(nn.Module):
  def __init__(self, num_cls):
    super().__init__()
    self.conv = nn.Sequential(
        *list(resnet18(weights=None).children())[:-2])

    self.fc = nn.Linear(512, num_cls)

  def forward(self, x):
    x = self.conv(x)
    x = torch.flatten(x, start_dim=1)
    logits = self.fc(x)

    return logits

In [4]:
!gdown 1-k2y76KAtvFxXDVPMDIzXtVa0sDgF0MQ

Downloading...
From (original): https://drive.google.com/uc?id=1-k2y76KAtvFxXDVPMDIzXtVa0sDgF0MQ
From (redirected): https://drive.google.com/uc?id=1-k2y76KAtvFxXDVPMDIzXtVa0sDgF0MQ&confirm=t&uuid=95704202-efec-4852-8916-27acf7d6709b
To: /content/resnet18_cifar10_model.pt
100% 44.8M/44.8M [00:00<00:00, 67.8MB/s]


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

CIFAR10_model_PATH = "/content/resnet18_cifar10_model.pt"
state_dict = torch.load(CIFAR10_model_PATH, map_location=device)


model = ResNet18(10).to(device)
model.load_state_dict(state_dict)


total = 0
correct = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test set: {100 * correct / total}')

Accuracy of the model on the test set: 83.87


# **Natural Evolutionary Strategies (NES)**

complete the functions in the cell below.

In [6]:
def estimate_grad(model, images, labels, search_variance, n, device):
    """
    NES Gradient Estimate

    inputs:
            - model: the target model (only used for computing the loss)
            - images: Tensor containing images. size: [number of images, 3, image_dim, image_dim]
            - labels: Tensor containing the original labels of images.
            - search_variance: sigma
            - n: number of samples to estimate the gradient

    outputs:
            - g: estimated gradients. has similar shape as images.

    Guide:  - Define g with initial value 0.
            - for n iterations do:
                - Define a tensor of random Gaussian noise with the input image shape (use torch.randn)
                - Divide this tensor by (sqrt(image_dim*image_dim*3)). (we do this due to the properties of gaussian distribution in high-dimensional space.)
                - Compute the gradient of the loss with finite difference method using the defined noise tensor.
                - Add the comupted value to g
    """

    #Your Code Goes Here (15 pt.)

    # Initialize the gradient estimate with zeros
    g = torch.zeros_like(images).to(device)
    batch_size = images.size(0)
    image_dim = images.size(2)  # Assuming square images
    noise_shape = images.shape

    # Calculate the normalization factor for the noise
    normalization_factor = (image_dim * image_dim * 3) ** 0.5

    # Set the model in evaluation mode
    model.eval()

    for _ in range(n):
      # Generate random Gaussian noise
      noise = torch.randn(noise_shape, device= device) / normalization_factor

      # Perturb the images with noise
      perturbed_images_plus = images + search_variance * noise
      perturbed_images_minus = images - search_variance * noise

      # Compute the loss for the perturbed images
      with torch.no_grad():
        logits_plus = model(perturbed_images_plus)
        logits_minus = model(perturbed_images_minus)

      # Compute the loss values
      loss_plus = F.cross_entropy(logits_plus, labels, reduction='none')
      loss_minus = F.cross_entropy(logits_minus, labels, reduction='none')

      # Compute the finite difference gradient estimate
      gradient_estimate = (loss_plus - loss_minus).view(batch_size, 1, 1, 1) * noise / (2 * search_variance)

      # Accumulate the gradient estimate
      g += gradient_estimate

    return g


def generate_attacks(model, images, labels, args, device):
    '''
    The process for generating blackbox adversarial examples. Implement for l_infty attack.

    inputs:
            - model: The target model
            - images: Tensor containing images of a batch. size: [batch_size, 3, image_dim, image_dim]
            - labels: Tensor containing the original labels of images. size: [batch_size, num_classes]

    outputs:
            - attacks: Must have the same shape as images.
            - total_queries: number of queries till a successful attack for each sample
            - success: Flag showing if each attack was successful or not.
    '''
    # Parameters
    epsilon = args['delta']  # Maximum perturbation
    alpha = args['epsilon']  # Step size
    max_queries = args['max_queries']
    num_samples = images.shape[0]

    total_queries = torch.zeros(num_samples).to(device)   #counter for number of queries. must be stopped for a sample when the attack for that sample is successful.
    success = torch.zeros(num_samples).to(device)          #boolean tensor indicating whether the attack is successfull or not.
    attacks = images.clone().to(device)


    #Here we want to generate PGD adversarial examples. we continue until we are out of queries.

    #Must Check not to have more queries than "max_queries" (Of course, you must update the value of total_queries)
    while  torch.any(total_queries < max_queries) and torch.any(success == 0):
        #Your Code Goes Here (15 pt)
        # Generate NES gradient estimates
        gradients = estimate_grad(model, attacks, labels, args['search_variance'], args['gradient_num_samples'], device)

        # Perform PGD update
        perturbations = alpha * torch.sign(gradients)
        attacks = attacks + perturbations
        attacks = torch.clamp(attacks, images - epsilon, images + epsilon)  # Project back into the epsilon ball
        attacks = torch.clamp(attacks, 0, 1)  # Ensure the perturbed image is still valid (pixel values between 0 and 1)

        # Evaluate the model on the perturbed images
        outputs = model(attacks)
        _, predicted = outputs.max(1)
        #_, original = labels.max(1)

        # Update success and query counts
        successful_attacks = (predicted != labels).float()
        success = torch.max(success, successful_attacks)
        total_queries += 1

        # Ensure that we stop updating total_queries for already successful attacks
        total_queries = torch.min(total_queries, max_queries * (success == 0) + total_queries * (success == 1))

    return attacks, total_queries, success

In [7]:
#  args = {'epsilon': 0.3, 'alpha': 0.01, 'max_queries': 100, 'search_variance': 0.1, 'n': 50}

In [8]:
# # Initialize model and move it to the appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

In [9]:
# attacks, total_queries, success = generate_attacks(model, images, labels, args)

# **Attack Report**

Report the success rate and average number of qureies for blackbox $l_∞$ PGD attacks for $\sigma \in \{0.001, 0.01\}$.



In [14]:
args = {'max_queries': 200, #maximum number of queries
        'search_variance': 0.01,  #sigma
        'epsilon': 0.01,    #step size in pgd
        'delta': 0.05,      #radius on which the attack must be projected
        'batch_size': 40,
        'gradient_num_samples': 15, #number of samples used for estimating the gradients
        }

In [11]:
testset = torchvision.datasets.CIFAR10(root='/content/cifar10/', train = False, download = True, transform = transforms.Compose([transforms.ToTensor(),]))
testloader = torch.utils.data.DataLoader(testset, batch_size = args['batch_size'], shuffle = False, num_workers = 2)


Files already downloaded and verified


In [15]:
###################################
#Your Code Goes Here (5 pt)
for i, data in enumerate(testloader):
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    attacks, total_queries, success = generate_attacks(model, images, labels, args, device)

    # Analyze the results
    print(f"Batch {i+1}")
    print(f"Total Queries: {total_queries}")
    print(f"Success Rate: {success.mean().item()}")

    # Optionally save the adversarial examples for further analysis
    # torch.save(attacks, f"adversarial_examples_batch_{i+1}.pth")

    if i == 0:  # Process only the first batch for demonstration purposes
        break

####################################

Batch 1
Total Queries: tensor([200., 200., 200., 200., 200., 200., 200., 200., 200., 200., 200., 200.,
        200., 200., 200., 200., 200., 200., 200., 200.])
Success Rate: 0.8999999761581421
